In [1]:
import sys

sys.path.insert(0, '..')

## Setup the model area

In [2]:
from morpheus.project.types.discretization import SpatialDiscretization
from morpheus.project.types.discretization.spatial import Rotation, Grid, Crs
from morpheus.project.types.geometry import Polygon, LineString

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.92, 50.964),
    (13.92, 50.966),
    (13.925, 50.966),
    (13.925, 50.964),
    (13.92, 50.964)
  ]]
)

rotation = Rotation(0)

grid = Grid.cartesian_from_polygon(polygon=polygon, n_cols=100, n_rows=50, rotation=rotation)
spatial_discretization = SpatialDiscretization.from_geometry_with_grid(
  geometry=polygon,
  grid=grid,
  crs=Crs.from_str('EPSG:4326')
)

## Setup Time Discretization

In [3]:
from morpheus.project.types.discretization.time.TimeUnit import TimeUnit
from morpheus.project.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState
from morpheus.project.types.discretization.time import TimeDiscretization
from datetime import datetime

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
  ]),
  time_unit=TimeUnit.days()
)

# Setup Model

In [4]:
from morpheus.project.types.Model import Model

model = Model.new()
model = model.with_updated_time_discretization(time_discretization)
model = model.with_updated_spatial_discretization(spatial_discretization=spatial_discretization)

In [5]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))

m.fit_bounds(m.get_bounds())
m

# Setup constant head boundary with two observations

In [6]:
from morpheus.project.types.boundaries.Observation import ObservationName
from morpheus.project.types.boundaries.RiverObservation import ObservationId, Conductance, RiverRawDataItem, RiverStage, \
  RiverbedBottom, RiverObservation
from morpheus.project.types.geometry import Point
from morpheus.project.types.boundaries.Boundary import BoundaryName, BoundaryCollection, RiverBoundary
from morpheus.project.types.layers import LayerId

layers_list = [LayerId.new()]

river_boundary = RiverBoundary.from_geometry(
  name=BoundaryName('river'),
  geometry=LineString([
    (13.92004, 50.96401),
    (13.92499, 50.966),
  ]),
  grid=grid,
  affected_layers=[model.layers[0].layer_id],
  observations=[
    RiverObservation(
      observation_id=ObservationId.new(),
      observation_name=ObservationName.default(),
      geometry=Point((13.92499, 50.966)),
      data=[
        RiverRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
          riverbed_bottom=RiverbedBottom.from_float(100),
          river_stage=RiverStage.from_float(105),
          conductance=Conductance.from_float(50),
        ),
        RiverRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
          riverbed_bottom=RiverbedBottom.from_float(100),
          river_stage=RiverStage.from_float(105),
          conductance=Conductance.from_float(50),
        ),
      ]
    ),
    RiverObservation(
      observation_id=ObservationId.new(),
      observation_name=ObservationName.default(),
      geometry=Point((13.92004, 50.96401)),
      data=[
        RiverRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
          riverbed_bottom=RiverbedBottom.from_float(95),
          river_stage=RiverStage.from_float(100),
          conductance=Conductance.from_float(100),
        ),
        RiverRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
          riverbed_bottom=RiverbedBottom.from_float(95),
          river_stage=RiverStage.from_float(100),
          conductance=Conductance.from_float(100),
        ),
      ]
    ),
  ]
)

boundary_collection = BoundaryCollection.new()
boundary_collection.add_boundary(river_boundary)

model = model.with_updated_boundaries(boundaries=boundary_collection)

m.add_child(folium.GeoJson(river_boundary.as_geojson()))
for observation in river_boundary.observations:
  m.add_child(folium.GeoJson(observation.as_geojson()))

m.fit_bounds(m.get_bounds())
m

In [7]:
river_boundary.observations

In [8]:
from morpheus.project.infrastructure.calculation.engines.modflow_2005.packages.RivPackageMapper import \
  calculate_riv_boundary_stress_period_data

result = calculate_riv_boundary_stress_period_data(
  spatial_discretization=spatial_discretization,
  time_discretization=time_discretization,
  layers=model.layers,
  riv_boundary=river_boundary,
)
result.to_dict()

## Calculate

In [9]:
from morpheus.project.types.calculation.CalculationProfile import CalculationProfile, CalculationEngineType
from morpheus.project.types.calculation.Calculation import Calculation

calculation_profile = CalculationProfile.new(CalculationEngineType.MF2005)
calculation = Calculation.new(model=model, calculation_profile=calculation_profile)

In [10]:
calculation = Calculation.from_dict(calculation.to_dict())
calculation.calculation_id

In [11]:
from morpheus.project.infrastructure.calculation.services.CalculationService import CalculationService

calculation_service = CalculationService.from_calculation(calculation)
calculation_service.calculate()

In [12]:
calculation_service.get_result()

In [13]:
head = calculation_service.read_head(0, 0)

In [14]:
import simplejson as json

json = json.dumps(head, ignore_nan=True)
json

In [15]:
calculation_service.read_drawdown(0, 0)

In [16]:
calculation_service.read_budget(0, False)

In [17]:
calculation_service.read_budget(0, True)

In [18]:
calculation_service.get_result()

In [19]:
import matplotlib.pyplot as plt

heads = calculation_service.read_head(0, 0)
plt.subplot(2, 1, 1)
plt.imshow(heads, cmap='jet_r')
plt.colorbar()
plt.show()